#  Laboratorio 5

##### Gladys Andrea Rodríguez Guerrero  -  20006107

In [1]:
#conda install -c conda-forge feature_engine
# conda install -c anaconda flask
# conda create -n tf tensorflow

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import tensorflow as tf

from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, Binarizer

from sklearn.pipeline import Pipeline

from feature_engine.imputation import(
    AddMissingIndicator,
    MeanMedianImputer,
    CategoricalImputer
)

from feature_engine.encoding import (
    RareLabelEncoder,
    OrdinalEncoder
)

from feature_engine.transformation import LogTransformer

from feature_engine.selection import DropFeatures
from feature_engine.wrappers import SklearnTransformerWrapper

import joblib

from keras.models import Model
import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import accuracy_score
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from keras.wrappers.scikit_learn import KerasRegressor
from keras.activations import relu, sigmoid
from keras.layers import Dense, Activation
from keras.models import Sequential

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

from sklearn.metrics import mean_squared_error

In [2]:
# pip install tensorflow

In [3]:
#Librería creada
import my_preprocessors as mypp 

In [4]:
data = pd.read_csv("train.csv")
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
cat_vars = [var for var in data.columns if data[var].dtype == 'O']
#cat_vars

#Agregamos variables que por definición son categóricas
cat_vars = cat_vars + ['Pclass'] + ['SibSp'] + ['Parch']
#cat_vars

data[cat_vars] = data[cat_vars].astype('O')

# Cantidad de variables categóricas
#len(cat_vars)

In [6]:
# Se separa data para train y test
X_train, X_test, y_train, y_test = train_test_split(
                        data.drop(['PassengerId', 'Survived'], axis=1),
                        data['Survived'],
                        test_size=0.15,
                        random_state=2021)

X_train.shape, X_test.shape

((757, 10), (134, 10))

In [7]:
# No se usa np.log por datos negatvos, genera error

#y_train = np.sqrt(np.power(y_train,2))
#y_test = np.sqrt(np.power(y_test,2))
#y_train
#y_test
#X_train

cat_vars_with_na = [var for var in cat_vars 
                        if X_train[var].isnull().sum() > 0]

X_train[cat_vars_with_na].isnull().mean().sort_values(ascending = False)

# variables tratadas con etiqueta de faltante por cantidad masiva de faltantes.
vars_with_missing_string = [var for var in cat_vars_with_na
                               if X_train[var].isnull().mean() > 0.2]

#variables tratadas con procedimiento por candiad adecuada de faltantes.
vars_freq_category = [var for var in cat_vars_with_na
                               if X_train[var].isnull().mean() <= 0.2]

#Faltantes con etiqueta missing
X_train[vars_with_missing_string] = X_train[vars_with_missing_string].fillna('Missing')
X_test[vars_with_missing_string] = X_test[vars_with_missing_string].fillna('Missing')

for var in vars_freq_category:
    mode = X_train[var].mode()[0]
    
    X_train[var].fillna(mode, inplace=True)
    X_test[var].fillna(mode, inplace=True)

cat_vars_with_na = [var for var in cat_vars 
                        if X_train[var].isnull().sum() > 0]

num_vars = [var for var in X_train.columns
               if var not in cat_vars and var != 'Survived']

# Numéricas con faltantes
num_vars_with_na = [var for var in num_vars
                       if X_train[var].isnull().sum() > 0]

for var in num_vars_with_na:
    mean_val = X_train[var].mean()
    
    X_train[var].fillna(mean_val, inplace=True)
    X_test[var].fillna(mean_val, inplace=True)

for var in ['Fare']:
    X_train[var] = np.sqrt(np.power(X_train[var],2))
    X_test[var] = np.sqrt(np.power(X_test[var],2))

X_test[vars_with_missing_string] = X_test[vars_with_missing_string].fillna('Missing')
X_test['Age'].isnull().sum()
#X_test[vars_with_missing_string]

## Configuración del Machine Learning Pipeline

#Variables categoricas con NA
CATEGORICAL_VARS_WITH_NA_MISSING = ['SibSp', 'Parch','Pclass']

#Variables numéricas con NA
NUMERICAL_VARS_WITH_NA = ['Age']


#Varibles para transformación logaritmia
NUMERICALS_LOG_VARS = ['Fare']

#Variables para hacer mapeo categorico por codificación ordinal
SEX_VARS = ['Sex']

#Variables categoricas a codificar sin ordinalidad
#CATEGORICAL_VARS = ['Cabin', 'Embarked', 'SibSp', 'Parch', 'Pclass']

#Mapeos de variables categoricas
SEX_MAPPINGS = {'male':1, 'female':0}

#Variables seleccionadas según análisis de Lasso
FEATURES = ['Sex', 'Age', 'Fare','SibSp', 'Parch','Pclass']

In [8]:
#Variables categoricas con NA
CATEGORICAL_VARS_WITH_NA_FREQUENT = []

#Variable categoricas con NA pero indicador de Missing
CATEGORICAL_VARS_WITH_NA_MISSING = []


#Variables numéricas con NA
NUMERICAL_VARS_WITH_NA = ['Age']


#Variables para binarización por sesgo fuerte
BINARIZE_VARS = ['Fare']

#Variables categoricas a codificar sin ordinalidad
CATEGORICAL_VARS = ['Sex']

#Variables seleccionadas según análisis de Lasso
FEATURES = [
    'Pclass',
    'Sex',  
    'Age', 
    'SibSp', 
    'Parch', 
    'Fare',
]

In [9]:
#Selección de variables para entrenamiento
X_train = X_train[FEATURES]
X_train

,Pclass,Sex,Age,SibSp,Parch,Fare
856,1,female,45.0,1,1,164.8667
685,2,male,25.0,1,2,41.5792
95,3,male,NaN,0,0,8.0500
419,3,female,10.0,0,2,24.1500
208,3,female,16.0,0,0,7.7500
...,...,...,...,...,...,...
621,1,male,42.0,1,0,52.5542
128,3,female,NaN,1,1,22.3583
57,3,male,28.5,0,0,7.2292
341,1,female,24.0,3,2,263.0000


In [10]:
#Seleccionamos variables para predicción
#X_test = pd.read_csv("test.csv")
#X_test = X_test[FEATURES]
#X_test

## Machine Learing PipeLine

In [11]:
Survived_pipeline = Pipeline([
    
    #============= IMPUTACIONES ===================#
    
    #1. Imputación de varaibles categoricas
    ('missing_imputation', 
         CategoricalImputer(imputation_method='missing', variables=CATEGORICAL_VARS_WITH_NA_MISSING)
    ),
    
    #2. Imputación de variables categoricas con NA basado en frequiencia.
    ('frequent_imputation', 
         CategoricalImputer(imputation_method='frequent', variables=CATEGORICAL_VARS_WITH_NA_FREQUENT)
    ),
    
    #3. Indicamos Faltante en variables numéricas para imputar
    ('missing_indicator', AddMissingIndicator(variables=NUMERICAL_VARS_WITH_NA)),
    
    #4. Imputación de mediana para variables categoricas
    ('mean_imputation', MeanMedianImputer(
        imputation_method='mean', variables=NUMERICAL_VARS_WITH_NA)
    ),
   
    #8. Binarización de Variables con Sesgo Fuerte
    ('binarizer', SklearnTransformerWrapper(
        transformer=Binarizer(threshold=0), variables=BINARIZE_VARS)
    ),
    
    #============ CODIFICACION DE VARIABLES CATEGORICAS NOMINALES ============
    
    ('rare_label_encoder', RareLabelEncoder(
        tol=0.01, n_categories=1, variables=CATEGORICAL_VARS)),
    
    ('categorical_encoder', OrdinalEncoder(
        encoding_method='ordered', variables=CATEGORICAL_VARS)),
    
    #=========== SCALER ==============
    ('scaler', MinMaxScaler()),
    
    #=========== ENTRENAMIENTO DEL MODELO ============
    ('Lasso', Lasso(alpha=0.01, random_state=2022)),
]) 

In [12]:
X_train

,Pclass,Sex,Age,SibSp,Parch,Fare
856,1,female,45.0,1,1,164.8667
685,2,male,25.0,1,2,41.5792
95,3,male,NaN,0,0,8.0500
419,3,female,10.0,0,2,24.1500
208,3,female,16.0,0,0,7.7500
...,...,...,...,...,...,...
621,1,male,42.0,1,0,52.5542
128,3,female,NaN,1,1,22.3583
57,3,male,28.5,0,0,7.2292
341,1,female,24.0,3,2,263.0000


In [13]:
Survived_pipeline.fit(X_train, y_train)

Pipeline(steps=[('missing_imputation', CategoricalImputer(variables=[])),
                ('frequent_imputation',
                 CategoricalImputer(imputation_method='frequent',
                                    variables=[])),
                ('missing_indicator', AddMissingIndicator(variables=['Age'])),
                ('mean_imputation',
                 MeanMedianImputer(imputation_method='mean',
                                   variables=['Age'])),
                ('binarizer',
                 SklearnTransformerWrapper(transformer=Binarizer(threshold=0),
                                           variables=['Fare'])),
                ('rare_label_encoder',
                 RareLabelEncoder(n_categories=1, tol=0.01, variables=['Sex'])),
                ('categorical_encoder', OrdinalEncoder(variables=['Sex'])),
                ('scaler', MinMaxScaler()),
                ('Lasso', Lasso(alpha=0.01, random_state=2022))])

In [14]:
#Seleccionamos variables para predicción
#X_test = pd.read_csv("test.csv")
X_test = X_test[FEATURES]
X_test

,Pclass,Sex,Age,SibSp,Parch,Fare
210,3,male,24.0,0,0,7.0500
876,3,male,20.0,0,0,9.8458
666,2,male,25.0,0,0,13.0000
819,3,male,10.0,3,2,27.9000
736,3,female,48.0,1,3,34.3750
...,...,...,...,...,...,...
41,2,female,27.0,1,0,21.0000
187,1,male,45.0,0,0,26.5500
46,3,male,NaN,1,0,15.5000
179,3,male,36.0,0,0,0.0000


cat_vars = [var for var in X_test.columns if X_test[var].dtype == 'O']
cat_vars

#Agregamos variables que por definición son categóricas
#cat_vars = cat_vars + ['Pclass'] + ['SibSp'] + ['Parch']
#cat_vars

data[cat_vars] = X_test[cat_vars].astype('O')

# Cantidad de variables categóricas
#len(cat_vars)

In [15]:
cat_vars 

['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked', 'Pclass', 'SibSp', 'Parch']

cat_vars_with_na = [var for var in cat_vars 
                        if X_test[var].isnull().sum() > 0]

# variables tratadas con etiqueta de faltante por cantidad masiva de faltantes.
vars_with_missing_string = [var for var in cat_vars_with_na
                               if X_test[var].isnull().mean() > 0.2]

# variables tratadas con procedimiento por candiad adecuada de faltantes.
vars_freq_category = [var for var in cat_vars_with_na
                               if X_test[var].isnull().mean() <= 0.2]

# Faltantes con etiqueta missing
X_train[vars_with_missing_string] = X_train[vars_with_missing_string].fillna('Missing')
X_test[vars_with_missing_string] = X_test[vars_with_missing_string].fillna('Missing')

for var in vars_freq_category:
    mode = X_test[var].mode()[0]
    
    X_train[var].fillna(mode, inplace=True)
    X_test[var].fillna(mode, inplace=True)
    
num_vars = [var for var in X_test.columns
               if var not in cat_vars and var != 'Survived']

# Numéricas con faltantes
num_vars_with_na = [var for var in num_vars
                       if X_test[var].isnull().sum() > 0]

for var in num_vars_with_na:
    mean_val = X_test[var].mean()
    
    X_train[var].fillna(mean_val, inplace=True)
    X_test[var].fillna(mean_val, inplace=True)

for var in ['Fare']:
    X_train[var] = np.sqrt(np.power(X_train[var],2))
    X_test[var] = np.sqrt(np.power(X_test[var],2))
    
sesgadas = ['Fare']

for var in sesgadas:
    X_train[var] = np.where(X_train[var] == 0, 0, 1)
    X_test[var] = np.where(X_test[var] == 0, 0, 1)

In [16]:
X_test

,Pclass,Sex,Age,SibSp,Parch,Fare
210,3,male,24.0,0,0,7.0500
876,3,male,20.0,0,0,9.8458
666,2,male,25.0,0,0,13.0000
819,3,male,10.0,3,2,27.9000
736,3,female,48.0,1,3,34.3750
...,...,...,...,...,...,...
41,2,female,27.0,1,0,21.0000
187,1,male,45.0,0,0,26.5500
46,3,male,NaN,1,0,15.5000
179,3,male,36.0,0,0,0.0000


In [17]:
X_test.isnull().sum()

Pclass     0
Sex        0
Age       25
SibSp      0
Parch      0
Fare       0
dtype: int64

In [18]:
preds = Survived_pipeline.predict(X_test)
preds

array([0.11145376, 0.11145376, 0.24118641, 0.11145376, 0.61922106,
       0.61922106, 0.87868637, 0.11145376, 0.24118641, 0.11145376,
       0.11145376, 0.37091907, 0.11145376, 0.11145376, 0.74895372,
       0.61922106, 0.11145376, 0.37091907, 0.11145376, 0.87868637,
       0.37091907, 0.11145376, 0.61922106, 0.61922106, 0.11145376,
       0.11145376, 0.11145376, 0.11145376, 0.87868637, 0.24118641,
       0.61922106, 0.11145376, 0.11145376, 0.11145376, 0.37091907,
       0.74895372, 0.11145376, 0.11145376, 0.37091907, 0.61922106,
       0.61922106, 0.11145376, 0.87868637, 0.11145376, 0.11145376,
       0.11145376, 0.11145376, 0.61922106, 0.37091907, 0.24118641,
       0.87868637, 0.74895372, 0.74895372, 0.61922106, 0.61922106,
       0.11145376, 0.24118641, 0.61922106, 0.74895372, 0.87868637,
       0.24118641, 0.24118641, 0.61922106, 0.61922106, 0.37091907,
       0.61922106, 0.11145376, 0.74895372, 0.11145376, 0.11145376,
       0.74895372, 0.74895372, 0.11145376, 0.61922106, 0.61922

In [19]:
#Survived_pipeline.transform(X_test)

In [20]:
mean_squared_error(np.exp(y_test), np.exp(preds), squared=False)

0.7632295183158672

In [21]:
X_test

,Pclass,Sex,Age,SibSp,Parch,Fare
210,3,male,24.0,0,0,7.0500
876,3,male,20.0,0,0,9.8458
666,2,male,25.0,0,0,13.0000
819,3,male,10.0,3,2,27.9000
736,3,female,48.0,1,3,34.3750
...,...,...,...,...,...,...
41,2,female,27.0,1,0,21.0000
187,1,male,45.0,0,0,26.5500
46,3,male,NaN,1,0,15.5000
179,3,male,36.0,0,0,0.0000


In [22]:
#Guardamos pipeline
joblib.dump(Survived_pipeline, 'Survived_pipeline.pkl')

['Survived_pipeline.pkl']

In [23]:
#X_train